In [1]:
#Load files and clean data

import pandas as pd
import numpy as np
import csv
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

#Columns to load
columns = ['id','company', 'origin', 'destination', 'departure', 'arrival', 'duration',
       'vehicle_type', 'vehicle_class', 'price', 'fare', 'seats']

#Load ticket sales file
renfe = pd.read_csv("thegurus-opendata-renfe-trips.csv", usecols = columns, index_col = ['id'])


#Drop rows with nulls in selected columns
renfe = renfe.dropna(subset=['price', 'origin', 'destination', 'departure', 'arrival'])

#Unify City names in one language
renfe.loc[renfe["origin"] == 'CASTELLO', "origin"] = 'CASTELLON'
renfe.loc[renfe["destination"] == 'CASTELLO', "destination"] = 'CASTELLON'

#Remove accent marks to unify city names
renfe["origin"] = renfe["origin"].str.replace('Á','A', regex = True)
renfe["destination"] = renfe["destination"].str.replace('Á','Á', regex = True)
renfe["origin"] = renfe["origin"].str.replace('É','É', regex = True)
renfe["destination"] = renfe["destination"].str.replace('É','E', regex = True)
renfe["origin"] = renfe["origin"].str.replace('Í','I', regex = True)
renfe["destination"] = renfe["destination"].str.replace('Í','I', regex = True)
renfe["origin"] = renfe["origin"].str.replace('Ó','O', regex = True)
renfe["destination"] = renfe["destination"].str.replace('Ó','O', regex = True)
renfe["origin"] = renfe["origin"].str.replace('Ú','U', regex = True)
renfe["destination"] = renfe["destination"].str.replace('Ú','U', regex = True)

#Unify travel types names
renfe.loc[renfe["vehicle_type"] == 'R. EXPRES', "vehicle_type"] = 'REG.EXP.'
renfe.loc[renfe["vehicle_type"] == 'AVE-AVE', "vehicle_type"] = 'AVE'

print("Rows Loaded: " + str(len(renfe.index)))

#Load coordinates file
cities = pd.read_csv("coordenadas.csv", sep = ';')



Rows Loaded: 29593111


In [2]:
# Companies

companies = np.sort(renfe.company.unique()).astype(str)

companies_unique = np.unique(companies)
companies_unique = pd.DataFrame(companies_unique, index = np.arange(len(companies_unique)))

companies_unique.reset_index(inplace=True)

#Rename Columns
companies_unique.columns = ['COMPANY_ID','COMPANY_DESC']

print(companies_unique)

   COMPANY_ID COMPANY_DESC
0           0        renfe


In [3]:
# Cities 

cities['Ciudad'] = cities['Ciudad'].str.upper() 

cities.lat.astype(float)
cities.Ing.astype(float)

origins = np.sort(renfe.origin.unique()).astype(str)

origins_unique = np.unique(np.char.upper(origins))

origins_unique = pd.DataFrame(origins_unique, index = np.arange(len(origins_unique)))

origins_unique.reset_index(inplace=True)

origins_unique.columns = ['CITY_ID','CITY_DESC']

cities_loc = origins_unique.merge(cities, left_on = 'CITY_DESC', right_on ='Ciudad', how = 'left').drop(columns = ['Ciudad'])

#Rename Columns
cities_loc.columns = ['CITY_ID','CITY_DESC','LATITUDE', 'LONGITUDE']

print(cities_loc)

    CITY_ID    CITY_DESC   LATITUDE  LONGITUDE
0         0     ALBACETE  38.973864  -1.853066
1         1     ALICANTE  38.409151  -0.512787
2         2    BARCELONA  41.381800   2.168500
3         3        CADIZ  36.500876  -6.268435
4         4    CASTELLON  39.986068  -0.036024
5         5  CIUDAD REAL  38.955416  -3.980987
6         6      CORDOBA  37.855096  -4.708674
7         7       CUENCA  40.217912  -1.991949
8         8       GIRONA  41.984200   2.823900
9         9      GRANADA  37.188627  -3.590778
10       10  GUADALAJARA  40.643624  -3.171878
11       11       HUESCA  42.148378  -0.412534
12       12         LEON  42.573567  -5.567159
13       13       LLEIDA  41.610900   0.641900
14       14       MADRID  40.489354  -3.682746
15       15       MALAGA  36.758541  -4.397172
16       16     PALENCIA  41.986007  -4.564922
17       17   PONFERRADA  42.502529  -6.571949
18       18      SEGOVIA  40.901942  -4.182989
19       19      SEVILLA  37.391411  -5.959178
20       20  

In [4]:
# Travel types 

travel_types = np.sort(renfe.vehicle_type.unique()).astype(str)

travel_types_unique = np.unique(np.char.upper(travel_types))

travel_types_unique = pd.DataFrame(travel_types_unique, index = np.arange(len(travel_types_unique)))

travel_types_unique.reset_index(inplace=True)

#Rename Columns
travel_types_unique.columns = ['TRAVEL_TYPE_ID','TRAVEL_TYPE_DESC']

print(travel_types_unique)

    TRAVEL_TYPE_ID TRAVEL_TYPE_DESC
0                0          ALTARIA
1                1            ALVIA
2                2          AV CITY
3                3            AVANT
4                4        AVANT-AVE
5                5         AVANT-LD
6                6         AVANT-MD
7                7              AVE
8                8        AVE-AVANT
9                9           AVE-LD
10              10           AVE-MD
11              11          AVE-TGV
12              12             AVLO
13              13          EUROMED
14              14        INTERCITY
15              15               LD
16              16         LD-AVANT
17              17           LD-AVE
18              18            LD-MD
19              19               MD
20              20         MD-AVANT
21              21           MD-AVE
22              22            MD-LD
23              23         REG.EXP.
24              24         REGIONAL
25              25        TORRE ORO
26              26        TR

In [5]:
# Fares

fares = np.sort(renfe.fare.unique()).astype(str)

fares_unique = pd.DataFrame(fares, index = np.arange(len(fares)))

fares_unique.reset_index(inplace=True)

#Rename Columns
fares_unique.columns = ['FARE_ID','FARE_DESC']

print(fares_unique)


    FARE_ID                FARE_DESC
0         0                    4x100
1         1               Adulto ida
2         2                   Básica
3         3          COD.PROMOCIONAL
4         4  Doble Familiar-Flexible
5         5                 Flexible
6         6               Grupos Ida
7         7      Individual-Flexible
8         8                     Mesa
9         9                    Promo
10       10                  Promo +
11       11                    YOVOY


In [6]:
# Classes

vehicle_class = np.sort(renfe.vehicle_class.unique()).astype(str)

vehicle_class_unique = pd.DataFrame(vehicle_class, index = np.arange(len(vehicle_class)))

vehicle_class_unique.reset_index(inplace=True)

#Rename Columns
vehicle_class_unique.columns = ['CLASS_ID','CLASS_DESC']

print(vehicle_class_unique)

    CLASS_ID                CLASS_DESC
0          0             Cama G. Clase
1          1              Cama Turista
2          2                Preferente
3          3    PreferenteSólo plaza H
4          4                   Turista
5          5    Turista - Turista Plus
6          6              Turista Plus
7          7    Turista Plus - Turista
8          8  Turista PlusSólo plaza H
9          9        Turista con enlace
10        10       TuristaSólo plaza H


In [7]:
# Ticket sales

#Merge Dataframes
renfe_company = renfe.merge(companies_unique, left_on = 'company', right_on ='COMPANY_DESC', how = 'left').drop(columns = ['COMPANY_DESC','company'])

renfe_vehicles = renfe_company.merge(travel_types_unique, left_on = 'vehicle_type', right_on ='TRAVEL_TYPE_DESC', how = 'left').drop(columns = ['TRAVEL_TYPE_DESC','vehicle_type'])

renfe_origins = renfe_vehicles.merge(origins_unique, left_on = 'origin', right_on ='CITY_DESC', how = 'left').drop(columns = ['CITY_DESC','origin'])

renfe_destinations = renfe_origins.merge(origins_unique, left_on = 'destination', right_on ='CITY_DESC', how = 'left').drop(columns = ['CITY_DESC','destination'])

renfe_fares = renfe_destinations.merge(fares_unique, left_on = 'fare', right_on ='FARE_DESC', how = 'left').drop(columns = ['FARE_DESC','fare'])

renfe_classes = renfe_fares.merge(vehicle_class_unique, left_on = 'vehicle_class', right_on ='CLASS_DESC', how = 'left').drop(columns = ['CLASS_DESC','vehicle_class'])


order_columns = ['COMPANY_ID', 'CITY_ID_x', 'CITY_ID_y', 'departure', 'arrival', 'duration', 'TRAVEL_TYPE_ID', 'CLASS_ID', 'FARE_ID', 'seats', 'price']

renfe_classes = renfe_classes.reindex(columns = order_columns)


db_columns = ['COMPANY_ID', 'ORIGIN_ID', 'DESTINATION_ID', 'DEPARTURE', 'ARRIVAL', 'TRAVEL_DURATION', 'TRAVEL_TYPE_ID', 'CLASS_ID', 'FARE_ID', 'SEATS', 'PRICE']

#Rename Columns
renfe_classes.columns =db_columns

renfe_classes['DEPARTURE'] = pd.to_datetime(renfe_classes['DEPARTURE'])
renfe_classes['ARRIVAL'] = pd.to_datetime(renfe_classes['ARRIVAL'])

print(renfe_classes.head())

   COMPANY_ID  ORIGIN_ID  DESTINATION_ID           DEPARTURE  \
0           0         14             2.0 2019-04-18 05:50:00   
1           0         14             2.0 2019-04-18 13:25:00   
2           0         14             2.0 2019-04-18 06:30:00   
3           0         14             2.0 2019-04-18 07:00:00   
4           0         14             2.0 2019-04-18 06:30:00   

              ARRIVAL  TRAVEL_DURATION  TRAVEL_TYPE_ID  CLASS_ID  FARE_ID  \
0 2019-04-18 08:55:00             3.08             7.0         2        9   
1 2019-04-18 16:24:00             2.98            11.0         4        5   
2 2019-04-18 09:20:00             2.83             7.0         4        9   
3 2019-04-18 09:30:00             2.50             7.0         6        9   
4 2019-04-18 09:20:00             2.83             7.0         4        9   

   SEATS   PRICE  
0    NaN   68.95  
1    NaN  107.70  
2    NaN   75.40  
3    NaN  106.75  
4    NaN   75.40  


In [9]:
# Database loading

from sqlalchemy.engine import create_engine

#Database Connection parameters
DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME =  
PASSWORD =  
HOST = 'localhost' 
PORT = 1521 
SERVICE = 'xe' 
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

#Engine creation
engine = create_engine(ENGINE_PATH_WIN_AUTH)
#Connection creation
con = engine.connect()

#Empty Database tables
con.execute('TRUNCATE TABLE lu_companies')
con.execute('TRUNCATE TABLE lu_travel_types')
con.execute('TRUNCATE TABLE lu_vehicle_classes')
con.execute('TRUNCATE TABLE lu_cities')
con.execute('TRUNCATE TABLE lu_fares')
con.execute('TRUNCATE TABLE dim_ticket_prices')

#Load Companies table
companies_unique.to_sql(con = engine, name = 'lu_companies', index = False, if_exists='append')
#Load Travel types table
travel_types_unique.to_sql(con = engine, name = 'lu_travel_types', index = False, if_exists='append')
#Load Classes table
vehicle_class_unique.to_sql(con = engine, name = 'lu_vehicle_classes', index = False, if_exists='append')
#Load Cities table
cities_loc.to_sql(con = engine, name = 'lu_cities', index = False, if_exists='append')
#Load Fares table
fares_unique.to_sql(con = engine, name = 'lu_fares', index = False, if_exists='append')

#Load Ticket sales table in 1M rows chunks
i = 1
while i <  len(renfe_classes.index):
    renfe_classes.iloc[i:i+1000000,:].to_sql(con = engine, name = 'dim_ticket_prices', index = False, if_exists='append')
    i=i+1000000

print('Loading complete')
    
# Close connection
con.close()

Loading complete
